In [62]:
%load_ext autoreload
# change import path 
import sys
sys.path.append('../../src')
# auto reload module


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from preprocessing.pre_processor import PreProcessor

In [37]:
# import roberta tokeizer fast
from transformers import RobertaTokenizerFast

# import torch
import torch

# import torch dataset

In [38]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

batch_size = 2
num_sentences = 24
max_sentence_length = 64
max_args_per_sentence = 10
max_arg_length = 16 
test_size = 0.1
frameaxis_dim = 10
name_tokenizer = 'roberta-base'
path_name_bert_model = 'roberta-base'
path_antonym_pairs = '../../data/axis/mft.json'
dim_names = ["virtue","vice"]
class_column_names = "Capacity and Resources;Crime and Punishment;Cultural Identity;Economic;External Regulation and Reputation;Fairness and Equality;Health and Safety;Legality, Constitutionality, Jurisdiction;Morality;Other;Policy Prescription and Evaluation;Political;Public Sentiment;Quality of Life;Security and Defense".split(";")

In [39]:
# Preprocess the input
preprocessor = PreProcessor(
    tokenizer,
    batch_size=batch_size,
    max_sentences_per_article=num_sentences,
    max_sentence_length=max_sentence_length,
    max_args_per_sentence=max_args_per_sentence,
    max_arg_length=max_arg_length,
    test_size=test_size,
    frameaxis_dim=frameaxis_dim,
    bert_model_name=name_tokenizer,
    name_tokenizer=name_tokenizer,
    path_name_bert_model=path_name_bert_model,
    path_antonym_pairs=path_antonym_pairs,
    dim_names=dim_names,
    class_column_names=class_column_names,
)

In [40]:
_, _, train_dataloader, test_dataloader = preprocessor.get_dataloader(
    "../../data/mfc/immigration_labeled_preprocessed.json",
    "json",
    dataframe_path={
        "srl": "../../data/srls/mfc/mfc_labeled.pkl",
        "frameaxis": "../../data/frameaxis/mfc/frameaxis_mft.pkl",
    },
    force_recalculate={
        "srl": False,
        "frameaxis": False,
    },
    sample_size=-1,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elias\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
sentence_idx = 3

print(tokenizer.decode(train_dataloader.dataset[502]["sentence_ids"][sentence_idx]))
print(tokenizer.decode(train_dataloader.dataset[502]["predicate_ids"][sentence_idx][0]))
print(tokenizer.decode(train_dataloader.dataset[502]["arg0_ids"][sentence_idx][0]))
print(tokenizer.decode(train_dataloader.dataset[502]["arg1_ids"][sentence_idx][0]))
print(class_column_names[np.argmax(train_dataloader.dataset[502]["labels"])])

James Wilson Marshall's discovery of gold at Coloma in 1848, of course, changed everything.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
 changed<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
James Wilson Marshall's discovery of gold at Coloma in 1848<pad><pad><pad>
 everything<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Cultural Identity


In [63]:
from model.slmuse_dlf.embeddings import SLMUSEEmbeddings

In [64]:
import torch
import torch.nn as nn

# Assuming the SLMUSEEmbeddings class is already defined as provided above

# Dummy logger class (to be replaced with the actual logger implementation)
class LoggerManager:
    @staticmethod
    def get_logger(name):
        class Logger:
            @staticmethod
            def debug(msg):
                print(f"DEBUG: {msg}")

            @staticmethod
            def info(msg):
                print(f"INFO: {msg}")

            @staticmethod
            def error(msg):
                print(f"ERROR: {msg}")
        
        return Logger()

# Initialize the embeddings model
model_name_or_path = "bert-base-uncased"
model_type = "bert-base-uncased"
slmuse_embeddings = SLMUSEEmbeddings(model_name_or_path, model_type, _debug=True)

# Create synthetic data
batch_size = 2
num_sentences = 3
max_sentence_length = 10
num_args = 2
max_arg_length = 5

# Random token IDs
sentence_ids = torch.randint(0, 100, (batch_size, num_sentences, max_sentence_length))
sentence_attention_masks = torch.ones((batch_size, num_sentences, max_sentence_length))

# Argument token IDs that do not match any sentence token IDs
predicate_ids = torch.randint(101, 200, (batch_size, num_sentences, num_args, max_arg_length))
arg0_ids = torch.randint(101, 200, (batch_size, num_sentences, num_args, max_arg_length))
arg1_ids = torch.randint(101, 200, (batch_size, num_sentences, num_args, max_arg_length))

# Perform forward pass
sentence_embeddings_avg, predicate_embeddings, arg0_embeddings, arg1_embeddings = slmuse_embeddings(
    sentence_ids, sentence_attention_masks, predicate_ids, arg0_ids, arg1_ids, mixed_precision="fp16"
)

# Print the outputs
print("Sentence Embeddings Avg:", sentence_embeddings_avg)
print("Predicate Embeddings:", predicate_embeddings)
print("Arg0 Embeddings:", arg0_embeddings)
print("Arg1 Embeddings:", arg1_embeddings)

# Check if arg_embeddings are all zeros as expected when no matching indices are found
assert torch.all(predicate_embeddings == 0), "Predicate embeddings should be all zeros."
assert torch.all(arg0_embeddings == 0), "Arg0 embeddings should be all zeros."
assert torch.all(arg1_embeddings == 0), "Arg1 embeddings should be all zeros."

print("Test passed: Argument embeddings are all zeros when no matching indices are found.")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationsh

Sentence Embeddings Avg: tensor([[[-0.1348, -0.1028,  0.3324,  ..., -0.4972,  0.8532,  0.0694],
         [-0.0515,  0.1625,  0.3968,  ..., -0.6832,  0.7695,  0.0592],
         [-0.0362,  0.0738,  0.3303,  ..., -0.4858,  0.8244,  0.0500]],

        [[-0.1378, -0.0636,  0.3563,  ..., -0.5113,  0.8731, -0.0124],
         [-0.0983,  0.0446,  0.3095,  ..., -0.5488,  0.8273,  0.0319],
         [-0.0792, -0.1050,  0.3861,  ..., -0.4678,  0.8429,  0.1195]]])
Predicate Embeddings: tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
Arg0

: 

In [1]:
import torch

# Sample data
batch_size = 2
num_sentences = 3
num_args = 4
embedding_dim = 5

# Create sample tensor with some zero embeddings
d_p = torch.randn(batch_size, num_sentences, num_args, embedding_dim)
d_a0 = torch.randn(batch_size, num_sentences, num_args, embedding_dim)
d_a1 = torch.randn(batch_size, num_sentences, num_args, embedding_dim)
d_fx = torch.randn(batch_size, num_sentences, embedding_dim)

# Introduce some zero embeddings
d_p[0, 1, :, :] = 0
d_a0[1, 2, :, :] = 0

batch_size, num_sentences, num_args, embedding_dim = d_p.shape

# Flatten the tensors
d_p_flatten = d_p.view(batch_size, num_sentences * num_args, embedding_dim)
d_a0_flatten = d_a0.view(batch_size, num_sentences * num_args, embedding_dim)
d_a1_flatten = d_a1.view(batch_size, num_sentences * num_args, embedding_dim)

# Create masks for non-zero embeddings
mask_p = (d_p_flatten.abs().sum(dim=-1) != 0).float()
mask_a0 = (d_a0_flatten.abs().sum(dim=-1) != 0).float()
mask_a1 = (d_a1_flatten.abs().sum(dim=-1) != 0).float()
mask_fx = (d_fx.abs().sum(dim=-1) != 0).float()

# Compute the masked mean for each descriptor
d_p_mean = (d_p_flatten * mask_p.unsqueeze(-1)).sum(dim=1) / torch.clamp(mask_p.sum(dim=1, keepdim=True), min=1)
d_a0_mean = (d_a0_flatten * mask_a0.unsqueeze(-1)).sum(dim=1) / torch.clamp(mask_a0.sum(dim=1, keepdim=True), min=1)
d_a1_mean = (d_a1_flatten * mask_a1.unsqueeze(-1)).sum(dim=1) / torch.clamp(mask_a1.sum(dim=1, keepdim=True), min=1)
d_fx_mean = (d_fx * mask_fx.unsqueeze(-1)).sum(dim=1) / torch.clamp(mask_fx.sum(dim=1, keepdim=True), min=1)

# Aggregate the masked means and then sum the result
y_hat_u = (d_p_mean + d_a0_mean + d_a1_mean + d_fx_mean) / 4

# Check the results
print("d_p_mean:", d_p_mean)
print("d_a0_mean:", d_a0_mean)
print("d_a1_mean:", d_a1_mean)
print("d_fx_mean:", d_fx_mean)
print("y_hat_u:", y_hat_u)

# Test if the means are correctly calculated and all-zero embeddings are masked
assert torch.all((d_p_flatten.abs().sum(dim=-1) == 0) == (mask_p == 0))
assert torch.all((d_a0_flatten.abs().sum(dim=-1) == 0) == (mask_a0 == 0))
assert torch.all((d_a1_flatten.abs().sum(dim=-1) == 0) == (mask_a1 == 0))
assert torch.all((d_fx.abs().sum(dim=-1) == 0) == (mask_fx == 0))

print("All zero embeddings correctly masked and means correctly calculated.")


d_p_mean: tensor([[-0.2429,  0.4436,  0.1822,  0.0661,  0.2601],
        [-0.5839, -0.1731, -0.0677, -0.4554, -0.0260]])
d_a0_mean: tensor([[-0.0737,  0.2785, -0.3001,  0.3141, -0.2100],
        [-0.2346, -0.0331, -0.2526,  0.3506,  0.1433]])
d_a1_mean: tensor([[ 0.0724, -0.2020, -0.0641, -0.1309,  0.0790],
        [-0.1767, -0.1864,  0.1031, -0.0578,  0.2027]])
d_fx_mean: tensor([[-0.6718,  0.4761, -1.1542,  0.2606,  0.2337],
        [-0.9713,  0.4796, -0.5544, -0.0870, -0.4772]])
y_hat_u: tensor([[-0.2290,  0.2491, -0.3340,  0.1275,  0.0907],
        [-0.4916,  0.0218, -0.1929, -0.0624, -0.0393]])
All zero embeddings correctly masked and means correctly calculated.
